# Flags

> Flags are custom ipython magics, that tell nbdev how to process cells in a notebook.

In this notebook, we test the flags defined in `flags.py` and show how you can start using them in your notebooks.

## Note: Special comment flags are still supported

Both comment and magic flags are currently supported and you can use both kinds in the same notebook.

Ideally, comment and magic flags would work the same but ... there is a difference when we `split_flags_and_code`.

## Using magic flags

In [ ]:
from nbdev import *

You can explore the available flags and their documentation without leaving your notebook

<img alt="Tab completion of nbdev magics" width="900" src="images/inspect_magics.png" />

Some flags print warnings if used incorrectly

<pre>
&percnt;nbdev_export bad module name
</pre>

<pre>UsageError: module_name "bad module name" must not contain whitespace</pre>

## How do comment flags correspond to magic flags?

| Comment flag                           | Magic flag                  |                                                                                |
|----------------------------------------|-----------------------------|--------------------------------------------------------------------------------|
| `default_exp`                          | `nbdev_default_export`      | Define the name of the module everything should be exported in                 |
| `exports`                              | `nbdev_export_and_show`     | Export and show code in the docs                                               |
| `exporti`                              | `nbdev_export_internal`     | Export but don’t show in docs and don’t add to `__all__`                       |
| `export`                               | `nbdev_export`              | Export but don’t show in docs                                                  |
| `hide_input`                           | `nbdev_hide_input`          | Do not show input of a test cell in docs                                       |
| `hide_output`                          | `nbdev_hide_output`         | Do not show output of a test cell in docs                                      |
| `hide`                                 | `nbdev_hide`                | Do not show a test cell or markdown in docs                                    |
| `default_cls_lvl`                      | `nbdev_default_class_level` | Define the default toc level of classes                                        |
| `collapse_output` or `collapse-output` | `nbdev_collapse_output`     | Inlcude output in the docs under a collapsable element                         |
| `collapse_show` or `collapse-show`     | `nbdev_collapse_input open` | Inlcude intput in the docs under a collapsable element that is open by default |
| `collapse`, `collapse_hide` or `collapse-hide` | `nbdev_collapse_input` | Inlcude intput in the docs under a collapsable element                      |

To add something to `__all__`, that is not picked automatically, you can assign a list of names to `_all_` or use the `nbdev_add2all` magic flag.

You can use `show_doc` to show documentation for an element, or `nbdev_show_doc` to show documentation for any number of elements.

## Tests

### We can test magics in ipython ...

In [ ]:
from nbdev.imports import *
from contextlib import redirect_stdout
import io

In [ ]:
if IN_IPYTHON:
    def test_flag_output(s, exp): test_eq(s.getvalue().strip(), exp.strip())
    s = io.StringIO()
    with redirect_stdout(s):
        %nbdev_export
        test_flag_output(s, '')
        %nbdev_export other.module
        test_flag_output(s, '')
        %nbdev_export bad module name
        test_flag_output(s, 'UsageError: module_name "bad module name" must not contain whitespace')

In [ ]:
if IN_IPYTHON:
    def test_flag_output(s, exp): test_eq(s.getvalue().strip(), exp.strip())
    s = io.StringIO()
    with redirect_stdout(s):
        %nbdev_fastai_test
        test_flag_output(s, '')
        %nbdev_fastai_test all
        test_flag_output(s, '')
        %nbdev_fastai_test not_all
        test_flag_output(s, 'UsageError: Invalid option "not_all". Usage `%nbdev_fastai_test [all]`')

### ... but we need less code to test in normal python

Note: We don't `del` the functions in `flags.py` so that we can test them here

In [ ]:
def test_flag_output(f, line, exp):
    "Test that `f(line)` prints `exp` to stdout"
    s = io.StringIO()
    with redirect_stdout(s): f(line)
    test_eq(s.getvalue().strip(), exp.strip())

In [ ]:
from nbdev.flags import nbdev_default_export

In [ ]:
test_flag_output(nbdev_default_export, '', 'UsageError: module_name is missing. Usage `%nbdev_default_export module_name`')
test_flag_output(nbdev_default_export, '  ', 'UsageError: module_name is missing. Usage `%nbdev_default_export module_name`')
test_flag_output(nbdev_default_export, 'bad module name', 'UsageError: module_name "bad module name" must not contain whitespace')
test_flag_output(nbdev_default_export, 'special.module', 
            'Cells will be exported to nbdev.special.module,\n' +
            'unless a different module is specified after an export flag: `%nbdev_export special.module`')

In [ ]:
from nbdev.flags import nbdev_export, nbdev_export_and_show, nbdev_export_internal

In [ ]:
for f in [nbdev_export, nbdev_export_and_show, nbdev_export_internal]:
    test_flag_output(f, '', '')
    test_flag_output(f, ' ', '')
    test_flag_output(f, 'special.module', '')
    test_flag_output(f, 'bad module name', 'UsageError: module_name "bad module name" must not contain whitespace')

In [ ]:
from nbdev.flags import nbdev_hide, nbdev_hide_input, nbdev_hide_output

In [ ]:
for f in [nbdev_hide, nbdev_hide_input, nbdev_hide_output]:
    test_flag_output(f, '', '')
    test_flag_output(f, 'anything', f'UsageError: Invalid option "anything". Usage `%{f.__name__}`')

In [ ]:
from nbdev.flags import nbdev_default_class_level

In [ ]:
for i in range(1,7):
    test_flag_output(nbdev_default_class_level, str(i), '')
test_flag_output(nbdev_default_class_level, '0', 'UsageError: Invalid class level "0". Usage `%nbdev_default_class_level [int between 1 and 6]`')
test_flag_output(nbdev_default_class_level, '7', 'UsageError: Invalid class level "7". Usage `%nbdev_default_class_level [int between 1 and 6]`')
test_flag_output(nbdev_default_class_level, 'not_a_num', 'UsageError: Invalid class level "not_a_num". Usage `%nbdev_default_class_level [int between 1 and 6]`')

In [ ]:
from nbdev.flags import nbdev_collapse_input, nbdev_collapse_output

In [ ]:
for f in [nbdev_collapse_input, nbdev_collapse_output]:
    test_flag_output(f, '', '')
    test_flag_output(f, ' open ', '')
    test_flag_output(f, 'open', '')
    test_flag_output(f, 'OPEN', f'UsageError: Invalid option "OPEN". Usage `%{f.__name__} [open]`')

In [ ]:
%nbdev_hide
from nbdev.flags import parse_line

In [ ]:
%nbdev_hide
for expected, line in [
        [['name'], 'name'],
        [['name', '"name2"'], 'name "name2"'],
        [['"name"', 'name2'], '"name", name2'],
        [['"name"', 'name2', 'name3'], '"name", name2 name3'],
        [['"name"', 'name2', 'name3'], ' , , "name", , name2 name3,,  '],
        [['"name"', 'name2'], '["name", name2]'],
        [['name', "'name2'"], " [ name, 'name2' ]"]]:
    test_eq(expected, parse_line(line))

In [ ]:
%nbdev_hide
from nbdev.flags import nbdev_add2all

In [ ]:
%nbdev_hide
fn = partial(nbdev_add2all, local_ns=globals())
test_flag_output(fn, '', 'UsageError: List of names is missing. Usage `%nbdev_add2all name_1, name_2`')
test_flag_output(fn, 'str int max', '')
test_flag_output(fn, 'str,int,max,', '')
test_flag_output(fn, 'str, int, max', '')
test_flag_output(fn, ' [ str ,,,"int" max ]', '')
test_flag_output(fn, '[str, "int", max ', 'UsageError: unexpected EOF while parsing (<string>, line 1)')
test_flag_output(fn, '"bad_name" max', '')
test_flag_output(fn, 'bad_name max', 'UsageError: name \'bad_name\' is not defined')
test_flag_output(fn, '"bad_name max', 'UsageError: EOL while scanning string literal (<string>, line 1)')

In [ ]:
%nbdev_hide
from nbdev.showdoc import nbdev_show_doc

In [ ]:
%nbdev_hide
fn = partial(nbdev_show_doc, local_ns=globals())
test_flag_output(fn, '', 'UsageError: List of names is missing. Usage `%nbdev_show_doc name_1, name_2`')
test_flag_output(fn, 'title_level=1', 'UsageError: List of names is missing. Usage `%nbdev_show_doc name_1, name_2`')
test_flag_output(fn, 'title_level=0 default_cls_level=2', 
"""UsageError: Invalid title_level "0". Usage `%nbdev_show_doc name_1 title_level=[int between 1 and 6]`
UsageError: List of names is missing. Usage `%nbdev_show_doc name_1, name_2`""")
test_flag_output(fn, 'title_level=1 default_cls_level=7', 
"""UsageError: Invalid default_cls_level "7". Usage `%nbdev_show_doc name_1 default_cls_level=[int between 1 and 6]`
UsageError: List of names is missing. Usage `%nbdev_show_doc name_1, name_2`""")

## Export -

In [ ]:
#hide
notebook2script()

Converted 00_export.ipynb.
Converted 01_sync.ipynb.
Converted 02_showdoc.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 05_merge.ipynb.
Converted 05a_conda.ipynb.
Converted 06_cli.ipynb.
Converted 07_clean.ipynb.
Converted 08_flag_tests.ipynb.
Converted 09_nbdev_callback_test.ipynb.
Converted 10_release.ipynb.
Converted 99_search.ipynb.
Converted index.ipynb.
Converted tutorial.ipynb.
